<a href="https://colab.research.google.com/github/jsp289/CS5901_Assignment2/blob/main/CS5901_assignment2_stage1_data_cleaning_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CS5901 - Assignment 2 - Stage 1**
*This .py file provides functions for data cleaning the CSV file*

### **Stage 1.1** - Import Data
*Import the CSV file from Git Repo and load into a Pandas data frame.*

In [1]:
# Initialize Google Drive when saved to G-Drive
#from google.colab import drive
#drive.mount('/content/drive')

import pandas as pd

def import_data(file_path):
  """
  This function imports data from a CSV file and load it into a Pandas DataFrame

  Args:
    file_path: location path of the CSV file
  Returns:
    df: a Pandas dataframe of the data in the CSV file
    else: print error message
  """

  #load CSV file in a Pandas dataframe using tab delimiter
  try:
    df_loaded = pd.read_csv(file_path, delimiter='\t')
    print(f"Data loaded successfully. Dimensions:{df_loaded.shape}")
    return df_loaded
  except Exception as e:
    print(f"Error loading data: {e}")
    return None

# File path of CSV file in Google-Drive
#file_path = '/content/drive/My Drive/P2data6332.csv'

#GitHub filepath
file_path = 'https://raw.githubusercontent.com/jsp289/CS5901_Assignment2/refs/heads/main/P2data6332.csv'

df =  import_data(file_path)
df.head()

Data loaded successfully. Dimensions:(482, 5)


,Level,T4,T3,T3adjusted,T4adjusted
0,5,8.1,2.1,2.008299,1.280579
1,5,8.7,NaN,2.056710,NaN
2,20,7.9,4.6,1.991632,1.663103
3,30,2.3,0.4,1.320006,0.736806
4,20,5.4,2.6,1.754411,1.375069


---
### Stages 1.2 - Remove Nonsensical Rows
*Here we drop negative values and outliers.*

In [2]:
def remove_nonsensical_rows(df):
  """
  This function removes negative values and outliers using the interquartile range method
  from the data frame generated in Stage 1.1

  Args:
    df: the Pandas dataframe generated in Stage 1.1
  Returns:
    df_cleaned: a Pandas dataframe with negative values and outliers removed
    nonsensical_data: a Pandas dataframe with negative values and outliers
  """

  # Identify negative values in columns
  negative_rows = df[(df[['T3','T4','T3adjusted','T4adjusted']]<0).any(axis=1)]

  #return negative_rows

#nonsensical_data = remove_nonsensical_rows(df)
#nonsensical_data.shape
#nonsensical_data.head()

  # Identify outliers with the IQR method
  Q1 = df[['T3','T4','T3adjusted','T4adjusted']].quantile(0.25)
  Q3 = df[['T3','T4','T3adjusted','T4adjusted']].quantile(0.75)
  IQR = Q3 - Q1

  #Calculate outlier boundaries
  lower_bound = Q1 - 1.5 * IQR
  upper_bound = Q3 + 1.5 * IQR

  #Isolate outlier rows
  outlier_rows = df.loc[((df[['T3','T4','T3adjusted','T4adjusted']] < lower_bound)|
                     (df[['T3','T4','T3adjusted','T4adjusted']] > upper_bound)).any(axis=1)]

  #return outlier_rows

#nonsensical_data = remove_nonsensical_rows(df)
#nonsensical_data.shape
#nonsensical_data.head()

  #Combine negative and outlier rows
  invalid_rows = pd.concat([negative_rows, outlier_rows]).drop_duplicates()

  #Drop invalid rows and duplicates
  df_cleaned = df.drop(invalid_rows.index).drop_duplicates()

  return df_cleaned, invalid_rows

df_cleaned, invalid_rows = remove_nonsensical_rows(df)
df_cleaned.shape
#invalid_rows.shape
df_cleaned.head()
#invalid_rows.head()


,Level,T4,T3,T3adjusted,T4adjusted
0,5,8.1,2.1,2.008299,1.280579
1,5,8.7,NaN,2.056710,NaN
2,20,7.9,4.6,1.991632,1.663103
3,30,2.3,0.4,1.320006,0.736806
4,20,5.4,2.6,1.754411,1.375069


---
### Stage 1.3 - Replace Missing Values
*Here we replace missing values in T3 and T4 by their average.*

In [3]:
def replace_missing_values(df):
  """
  This function replaces missing values in T3,T4, T3adjusted, T4adjusted by their average
  from the cleaned dataframe generated in Stage 1.2

  Args:
    df: the cleaned Pandas dataframe generated in Stage 1.2
  Returns:
    df_filled: a cleaned Pandas dataframe where the missing values in T3 and T4 are replaced
    by their average.
  """

  #Calculate the averages of columns T3,T4, T3adjusted, T4adjusted
  col_means = df.groupby("Level")[["T3","T4","T3adjusted","T4adjusted"]].mean()

  #Replace missing values in Stage 1.2 with corresponding values from col_means
  df_filled = df.copy()
  df_filled["T3"] = df_filled.apply(lambda row: col_means.loc[row["Level"],"T3"] if pd.isna(row["T3"]) else row["T3"], axis=1)
  df_filled["T4"] = df_filled.apply(lambda row: col_means.loc[row["Level"],"T4"] if pd.isna(row["T4"]) else row["T4"], axis=1)
  df_filled["T3adjusted"] = df_filled.apply(lambda row: col_means.loc[row["Level"],"T3adjusted"] if pd.isna(row["T3adjusted"]) else row["T3adjusted"], axis=1)
  df_filled["T4adjusted"] = df_filled.apply(lambda row: col_means.loc[row["Level"],"T4adjusted"] if pd.isna(row["T4adjusted"]) else row["T4adjusted"], axis=1)

  return df_filled

df_filled = replace_missing_values(df_cleaned)
df_filled.head()



,Level,T4,T3,T3adjusted,T4adjusted
0,5,8.1,2.10000,2.008299,1.280579
1,5,8.7,4.41998,2.056710,1.467960
2,20,7.9,4.60000,1.991632,1.663103
3,30,2.3,0.40000,1.320006,0.736806
4,20,5.4,2.60000,1.754411,1.375069


---
### Stage 1.4 - Calculate Descriptive Statistics Without Pandas
*Here we compute descriptive statistics manually and compare results against and compare against the pd.describe() method.*

In [4]:
def calculate_stats(df):
  """
  This function computes descriptive statistics manually and compare results against and compare against the pd.describe() method
  from the filled dataframe generated in Stage 1.3

  Args:
    df: the filled Pandas dataframe generated in Stage 1.3
  Returns:
    manual_stats: a Pandas dataframe with descriptive statistics computed manually
    pd_stats: a Pandas dataframe with descriptive statistics computed using pd.describe()
  """
  #Create the dataframe for manual_stats
  manual_stats = pd.DataFrame()

  #Compute stats manually and populate manual_stats dataframe
  manual_stats["count"] = df.count()
  manual_stats["mean"] = df.mean()
  manual_stats["std"] = df.std()
  manual_stats["min"] = df.min()
  manual_stats["25%"] = df.quantile(0.25)
  manual_stats["50%"] = df.quantile(0.5)
  manual_stats["75%"] = df.quantile(0.75)
  manual_stats["max"] = df.max()

  #Transpose manual_stats to match Pandas formatting
  manual_stats = manual_stats.transpose()
  pd_stats = df.describe()

  print("The two dataframes are equal (T or F):", manual_stats.equals(pd_stats))
  return manual_stats, pd_stats


manual_stats = calculate_stats(df_filled)
manual_stats







The two dataframes are equal (T or F): False


(            Level          T4          T3  T3adjusted  T4adjusted
 count  384.000000  384.000000  384.000000  384.000000  384.000000
 mean    28.268229   12.970317   11.609370    2.192149    2.038168
 std     51.336533    9.345622    9.910619    0.639976    0.679945
 min      5.000000  -13.600000   -4.500000   -2.386966   -1.650964
 25%     10.000000    5.905058    4.419980    1.807483    1.557038
 50%     20.000000   11.071429    8.000000    2.223730    1.995816
 75%     20.000000   18.425000   15.511765    2.641206    2.461178
 max    400.000000   41.400000   42.300000    3.459395    3.484283,
             Level          T4          T3  T3adjusted  T4adjusted
 count  384.000000  384.000000  384.000000  384.000000  384.000000
 mean    28.268229   12.970317   11.609370    2.192149    2.038168
 std     51.336533    9.345622    9.910619    0.639976    0.679945
 min      5.000000  -13.600000   -4.500000   -2.386966   -1.650964
 25%     10.000000    5.905058    4.419980    1.807483    1.5

---
### Stage 1.5 - Identify Duplicate Rows
*Here we identify and confirm the existence of duplicate rows in df_filled*


In [5]:
def duplicate_rows(df):
  """
  This function identifies and confirms the existence of duplicate rows in df_filled and drops them.

  Args:
    df: the df_filled Pandas dataframe generated in Stage 1.3
  Returns:
    duplicate_data: a Pandas dataframe with duplicate rows
    df_filled_no_duplicates: a Pandas dataframe with duplicate rows removed
  """

  #Create dataframe for duplicate rows
  duplicate_data = df_filled[df_filled.duplicated()]
  print(duplicate_data)
  print(f"Number of duplicate rows: {duplicate_data.shape[0]}")

  #Create dataframe without duplicate rows
  df_filled_no_duplicates = df_filled.drop_duplicates()
  print(f"Number of rows after dropping duplicates: {df_filled_no_duplicates.shape[0]}")

  return duplicate_data, df_filled_no_duplicates


duplicate_data, df_filled_no_duplicates = duplicate_rows(df_filled)
df_filled_no_duplicates.head()



Empty DataFrame
Columns: [Level, T4, T3, T3adjusted, T4adjusted]
Index: []
Number of duplicate rows: 0
Number of rows after dropping duplicates: 384


,Level,T4,T3,T3adjusted,T4adjusted
0,5,8.1,2.10000,2.008299,1.280579
1,5,8.7,4.41998,2.056710,1.467960
2,20,7.9,4.60000,1.991632,1.663103
3,30,2.3,0.40000,1.320006,0.736806
4,20,5.4,2.60000,1.754411,1.375069
